<a href="https://colab.research.google.com/github/descartesmbogning/phage-therapy-patents-analysis/blob/main/3_patent_Innovation_Hotspots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk matplotlib wordcloud scikit-learn


In [ ]:
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import plotly.express as px
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Downloads
nltk.download('wordnet')
nltk.download('stopwords')

# Initialize your stopwords set with wordcloud's stopwords
s = set(STOPWORDS)

# Get stopwords for all languages in NLTK
languages = stopwords.fileids()
for language in languages:
    s.update(stopwords.words(language))

# Add any additional stopwords if needed
s.add('amp')

# If you really need the resulting stopwords as a list:
stopwords_list = list(s)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Load your dataset (Assuming it's a CSV with a column 'abstract' for patent abstracts)

# import data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
data = pd.read_csv ('/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/patent-phage-therapy-lens-23102023-tiab.csv')
#data = pd.read_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/topic_probability_patent_therapy_23102023_10.pkl').reset_index()
pd.set_option('display.max_columns', None)
print(len(data))
data.head()

# data preprocessing

In [ ]:
data['Application Date'] = pd.to_datetime(data['Application Date'])
data['Earliest Priority Date'] = pd.to_datetime(data['Earliest Priority Date'])

In [ ]:
data['Application Year'] = data['Application Date'].dt.year
data['Earliest Priority Year'] = data['Earliest Priority Date'].dt.year

In [ ]:
# Convert 'Publication Year' to integers
data['Publication Year'] = data['Publication Year'].astype('Int64')
data['Application Year'] = data['Application Year'].astype('Int64')
data['Earliest Priority Year'] = data['Earliest Priority Year'].astype('Int64')

In [ ]:
def group_years_in_ranges(df, col_name):
    bins = list(range(1960, 2026, 5))
    labels = [f"{i}-{i+4}" for i in bins[:-1]]
    return pd.cut(data['Application Year'], bins=bins, labels=labels, right=False)


In [ ]:
data['Application_Year_Range'] = group_years_in_ranges(data, 'Application Year')


In [ ]:
data['IPCR Classifications']

In [ ]:
import pandas as pd

# Define the function to process IPCR codes
def process_ipcr_sections(ipcr_codes):
    # Check if ipcr_codes is a string (which means it's not NaN or None)
    if isinstance(ipcr_codes, str):
        # Extract the first letter of each code, remove duplicates by converting to a set
        unique_initials = sorted(set(code[0] for code in ipcr_codes.split(';;') if code))
        # Join the unique initials back together
        return ';;'.join(unique_initials)
    else:
        # If it's not a string, just return an empty string or NaN
        return ''

# Assuming 'data' is your DataFrame
# Apply the function to the 'IPCR Classifications' column
data['IPCR Sections'] = data['IPCR Classifications'].apply(process_ipcr_sections)

# Print the DataFrame to verify the results
data[['IPCR Classifications', 'IPCR Sections']]


In [ ]:
import pandas as pd

# Define the function to process IPCR codes
def process_ipcr_class(ipcr_codes):
    # Check if ipcr_codes is a string (which means it's not NaN or None)
    if isinstance(ipcr_codes, str):
        # Extract the first letter of each code, remove duplicates by converting to a set
        unique_initials = sorted(set(code[:3] for code in ipcr_codes.split(';;') if code))
        # Join the unique initials back together
        return ';;'.join(unique_initials)
    else:
        # If it's not a string, just return an empty string or NaN
        return ''

# Assuming 'data' is your DataFrame
# Apply the function to the 'IPCR Classifications' column
data['IPCR Class'] = data['IPCR Classifications'].apply(process_ipcr_class)

# Print the DataFrame to verify the results
data[['IPCR Classifications', 'IPCR Class']]


In [ ]:
import pandas as pd

# Define the function to process IPCR codes
def process_ipcr_subclass(ipcr_codes):
    # Check if ipcr_codes is a string (which means it's not NaN or None)
    if isinstance(ipcr_codes, str):
        # Extract the first letter of each code, remove duplicates by converting to a set
        unique_initials = sorted(set(code[:4] for code in ipcr_codes.split(';;') if code))
        # Join the unique initials back together
        return ';;'.join(unique_initials)
    else:
        # If it's not a string, just return an empty string or NaN
        return ''

# Assuming 'data' is your DataFrame
# Apply the function to the 'IPCR Classifications' column
data['IPCR Subclass'] = data['IPCR Classifications'].apply(process_ipcr_subclass)

# Print the DataFrame to verify the results
data[['IPCR Classifications', 'IPCR Subclass']]


In [ ]:
data['Jurisdiction'].unique()

In [ ]:
import pandas as pd

# Assuming `data` is your DataFrame and has a column named 'Jurisdiction'

jurisdiction_codes = {
    'RU': 'Russia',
    'JP': 'Japan',
    'AU': 'Australia',
    'US': 'United States',
    'CN': 'China',
    'CA': 'Canada',
    'WO': 'World Intellectual Property Org.',
    'EP': 'European Patent Office',
    'SG': 'Singapore',
    'KR': 'South Korea',
    'NZ': 'New Zealand',
    'GB': 'United Kingdom',
    'GE': 'Georgia',  # or 'Germany' for historical East Germany
    'DE': 'Germany',
    'CZ': 'Czech Republic',
    'MX': 'Mexico',
    'TN': 'Tunisia',
    'ES': 'Spain',
    'IN': 'India',
    'EA': 'Eurasian Patent Org.',
    'TW': 'Taiwan',
    'UA': 'Ukraine',
    'MY': 'Malaysia',
    'SU': 'Russia',
    'FR': 'France',
    'CH': 'Switzerland',
    'PH': 'Philippines',
    'SK': 'Slovakia',
    'NL': 'Netherlands',
    'LV': 'Latvia',
    'ZA': 'South Africa',
    'PL': 'Poland',
    'BG': 'Bulgaria',
}

# Replace the codes with names in the 'Jurisdiction' column
data['Jurisdiction_name'] = data['Jurisdiction'].map(jurisdiction_codes).fillna(data['Jurisdiction'])

#data[['Jurisdiction', 'Jurisdiction_name']]

In [ ]:
import pandas as pd

# Your existing DataFrame is 'data', and it contains a column 'Jurisdiction'

# Map of jurisdiction codes to their respective continent or region names
jurisdiction_to_region = {
    # Western Europe
    'GB': 'Europe', 'DE': 'Europe', 'FR': 'Europe',
    'CH': 'Europe', 'NL': 'Europe', 'ES': 'Europe',

    # Eastern Europe
    'RU': 'Europe', 'CZ': 'Europe', 'UA': 'Europe',
    'SK': 'Europe', 'LV': 'Europe', 'PL': 'Europe',
    'GE': 'Europe', 'BG': 'Europe',

    # Rest of the regions
    'JP': 'Asia', 'AU': 'Oceania', 'US': 'North America', 'CN': 'Asia',
    'CA': 'North America', 'WO': 'International', 'EP': 'Europe', 'SG': 'Asia',
    'KR': 'Asia', 'NZ': 'Oceania', 'MX': 'North America', 'TN': 'Africa',
    'IN': 'Asia', 'EA': 'Eurasia', 'TW': 'Asia', 'MY': 'Asia',
    'SU': 'Eurasia', 'PH': 'Asia', 'ZA': 'Africa'
}

# Create a new column 'Region' by mapping the 'Jurisdiction' column using the dictionary
data['Jurisdiction_Region'] = data['Jurisdiction'].map(jurisdiction_to_region).fillna(data['Jurisdiction'])

data[['Jurisdiction', 'Jurisdiction_name', 'Jurisdiction_Region']]


In [ ]:
data['Jurisdiction_name'].unique()

In [ ]:
data['Jurisdiction_Region'].unique()

In [ ]:
data.info()

### clean applicants and inventors

In [ ]:
data[['Applicants','Inventors']]

In [ ]:
#data_topic = pd.read_csv ('/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/patent-phage-therapy-lens-23102023-tiab.csv')
applicant_clean = pd.read_excel(f'/content/drive/Othercomputers/My laptop/D:/PROJECT/YORK UNIVERSITY/BACTERIOPHAGES/MANUSCRIPT/applicant-classification_clean.xlsx')
pd.set_option('display.max_columns', None)
print(len(applicant_clean))
applicant_clean.head()

In [ ]:
#data_topic = pd.read_csv ('/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/patent-phage-therapy-lens-23102023-tiab.csv')
inventor_clean = pd.read_excel('/content/drive/Othercomputers/My laptop/D:/PROJECT/YORK UNIVERSITY/BACTERIOPHAGES/MANUSCRIPT/inventor_list_clean2.xlsx')
pd.set_option('display.max_columns', None)
print(len(inventor_clean))
inventor_clean.head()

In [ ]:
def clean_entities(df1, df2, col_old, col_clean, col_name):
    # Create a dictionary from df2 for mapping
    disease_dict = df2.set_index(col_old)[col_clean].to_dict()

    # Function to clean a single cell
    #def clean_cell(cell):
    def clean_cell(cell):
        if isinstance(cell, str):
            diseases = cell.split(';;')
            cleaned = list(set([disease_dict.get(disease, disease) for disease in diseases if isinstance(disease_dict.get(disease, disease), str)]))
            return ';;'.join(cleaned)
        else:
            return cell


    # Apply the function to each cell in the 'Diseases_lemat_stop2' column
    df1[col_name] = df1[col_old].apply(clean_cell)

    return df1

# Use the function
#df1_clean = clean_diseases(df1, df2)


#### clean inventors

In [ ]:
df1 = data
df2 =  inventor_clean
col_old =  'Inventors'
col_clean =  'inventors_clean'
col_name = 'Inventors_clean'


data = clean_entities(df1, df2, col_old, col_clean, col_name)#.reset_index()
data

#### clean applicants

In [ ]:
df1 = data
df2 =  applicant_clean
col_old =  'Applicants'
col_clean =  'applicants_clean'
col_name = 'Applicants_clean'


data = clean_entities(df1, df2, col_old, col_clean, col_name)#.reset_index()
data

#### add applicants countries

In [ ]:
df1 = data
df2 =  applicant_clean
col_old =  'Applicants'
col_clean =  'country_clean'
col_name = 'Applicants_countries'

data = clean_entities(df1, df2, col_old, col_clean, col_name)#.reset_index()
data

#### clean applicants not individual

In [ ]:
df1 = data
df2 =  applicant_clean
col_old =  'Applicants'
col_clean =  'applicants_clean_not_individual'
col_name = 'applicants_clean_not_individual'


data = clean_entities(df1, df2, col_old, col_clean, col_name)#.reset_index()
data

,#,Jurisdiction,Kind,Display Key,Lens ID,Publication Date,Publication Year,Application Number,Application Date,Priority Numbers,Earliest Priority Date,Title,Abstract,Applicants,Inventors,Owners,URL,Document Type,Has Full Text,Cites Patent Count,Cited by Patent Count,Simple Family Size,Extended Family Size,Sequence Count,CPC Classifications,IPCR Classifications,US Classifications,NPL Citation Count,NPL Resolved Citation Count,NPL Resolved Lens ID(s),NPL Resolved External ID(s),NPL Citations,Legal Status,Topic N°,Application Year,Earliest Priority Year,Application_Year_Range,IPCR Sections,IPCR Class,IPCR Subclass,Jurisdiction_name,Jurisdiction_Region,Inventors_clean,Applicants_clean,applicants_clean_not_individual
0,1,RU,C2,RU 2214829 C2,069-390-611-717-343,2003-10-27,2003,RU 2001129920 A,2001-11-05,RU 2001129920 A,2001-11-05,METHOD FOR TUBERCULOSIS PHAGOTHERAPY,"FIELD: medicine, phthisiology. SUBSTANCE: meth...",NOVOSIB NII T;;UBERKULEZA,KURUNOV JU N;;KURUNOVA N N;;SHATALOVA N D,NaN,https://lens.org/069-390-611-717-343,Granted Patent,no,0,4,1,1,0,NaN,A61K35/76;;A61P31/06,NaN,0,0,NaN,NaN,NaN,EXPIRED,Topic 2,2001,2001,2000-2004,A,A61,A61K;;A61P,Russia,Europe,SHATALOVA N D;;KURUNOVA N N;;KURUNOV JU N,UBERKULEZA;;NOVOSIB NII T,NOVOSIB NII T
1,2,JP,A,JP 2016104011 A,114-759-028-882-736,2016-06-09,2016,JP 2015243657 A,2015-12-14,US 38401510 P,2010-09-17,"ANTIBACTERIAL PHAGES, PHAGE PEPTIDES AND USE M...",PROBLEM TO BE SOLVED: To provide novel bacteri...,TECNIFAR-INDUSTRIA TECNICA FARM S A;;TECHNOPHA...,MIGUEL ANGELO DA COSTA GARCIA;;CARLOS JORGE SO...,NaN,https://lens.org/114-759-028-882-736,Patent Application,no,1,1,48,48,1300,C12N2795/00032;;A61K31/505;;A61K31/7056;;A61K3...,C12N7/00;;A61K35/76;;A61P31/04;;C12N15/09,NaN,0,0,NaN,NaN,NaN,ACTIVE,Topic 3,2015,2010,2015-2019,A;;C,A61;;C12,A61K;;A61P;;C12N,Japan,Asia,CARLOS JORGE SOUSA DE SAO JOSE;;ANA RAQUEL MAR...,TECNIFAR INDUSTRIA TECNICA FARMACEUTICA SA;;Te...,TECNIFAR INDUSTRIA TECNICA FARMACEUTICA SA;;Te...
2,3,AU,A1,AU 2015/255318 A1,094-982-928-461-863,2015-12-03,2015,AU 2015/255318 A,2015-11-13,AU 2015/255318 A;;AU 2010/211456 A;;US 2183450...,2009-02-06,"Antibacterial phage, phage peptides and method...",The present invention is directed to the field...,TECNIFAR INDÚSTRIA TÉCNICA FARMACÉUTICA S A;;T...,DA COSTA GARCIA MIGUEL ÂNGELO;;SOUSA DE SÃO JO...,"TECHNOPHAGE, INVESTIGACAO E DESENVOLVIMENTO EM...",https://lens.org/094-982-928-461-863,Patent Application,no,0,0,3,45,0,NaN,A61K35/76;;C12N9/14;;G01N33/569,NaN,0,0,NaN,NaN,NaN,ACTIVE,Topic 3,2015,2009,2015-2019,A;;C;;G,A61;;C12;;G01,A61K;;C12N;;G01N,Australia,Oceania,CARLOS JORGE SOUSA DE SAO JOSE;;FERREIRA LLORE...,TECNIFAR INDUSTRIA TECNICA FARMACEUTICA SA;;Te...,TECNIFAR INDUSTRIA TECNICA FARMACEUTICA SA;;Te...
3,4,US,A1,US 2016/0022747 A1,167-908-891-122-112,2016-01-28,2016,US 201514852112 A,2015-09-11,US 201514852112 A;;US 201113148009 A;;PT 20100...,2009-02-06,"Antibacterial Phage, Phage Peptides and Method...",The present invention is directed to the field...,TECHNOPHAGE INVESTIGACAO E DESENVOLVIMENTO EM ...,DA COSTA GARCIA MIGUEL ANGELO;;SOUSA DE SAO JO...,TECHNOPHAGE INVESTIGACAO E DESENVOLVIMENTO EM ...,https://lens.org/167-908-891-122-112,Patent Application,yes,0,1,42,45,816,A61K38/00;;A61K45/06;;C07K14/005;;C12N7/00;;C1...,A61K35/76;;A61K9/00;;C12N7/00,NaN,0,0,NaN,NaN,NaN,ACTIVE,NaN,2015,2009,2015-2019,A;;C,A61;;C12,A61K;;C12N,United States,North America,CARLOS JORGE SOUSA DE SAO JOSE;;FERREIRA LLORE...,TECNIFAR INDUSTRIA TECNICA FARMACEUTICA SA;;Te...,TECNIFAR INDUSTRIA TECNICA FARMACEUTICA SA;;Te...
4,5,AU,A1,AU 2017/204331 A1,112-018-476-081-94X,2017-07-13,2017,AU 2017/204331 A,2017-06-26,AU 2017/204331 A;;AU 2015/255313 A;;AU 2011/30...,2011-09-19,"Antibacterial phage, phage peptides and method...",The present invention is directed to the field...,TECHNOPHAGE INVESTIGACAO E DESENVOLVIMENTO EM ...,DA COSTA GARCIA MIGUEL ANGELO;;SOUSA DE SAO JO...,NaN,https://lens.org/112-018-476-081-94X,Patent Application,no,0,3,

#### add applicants region

In [ ]:
df1 = data
df2 =  applicant_clean
col_old =  'Applicants'
col_clean =  'region_clean'
col_name = 'Applicants_regions'

data = clean_entities(df1, df2, col_old, col_clean, col_name)#.reset_index()
data

#### add applicants categories

In [ ]:
df1 = data
df2 =  applicant_clean
col_old =  'Applicants'
col_clean =  'applicant_category'
col_name = 'Applicants_categories'

data = clean_entities(df1, df2, col_old, col_clean, col_name)#.reset_index()
data

In [ ]:
data = data.drop(['Applicants','Inventors'], axis=1).rename(columns={'Applicants_clean': 'Applicants', 'Inventors_clean': 'Inventors'})

In [ ]:
data

# Innovation Hotspots:
•	Objective: Identify regions or countries leading in phage therapy innovations.

    o	Research Question: Which countries or regions have the highest number of patents related to phage therapy, and what might be driving this innovation?


In [ ]:
df= data['Jurisdiction_Region'].value_counts().to_frame()
df['Jurisdiction_Region_prop'] = df['Jurisdiction_Region'] / df['Jurisdiction_Region'].sum()
df


## Nested Pie Chart: Jurisdiction Counts by Region

In [ ]:
data.groupby(['Jurisdiction_Region', 'Jurisdiction_name']).size().reset_index(name='counts').sort_values(by=['Jurisdiction_Region', 'counts'], ascending=[True, False])


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


# Let's create the DataFrame from the provided data
#df = data.groupby(['Jurisdiction_Region', 'Jurisdiction_name']).size().reset_index(name='counts')
df = data.groupby(['Jurisdiction_Region', 'Jurisdiction_name']).size().reset_index(name='counts').sort_values(by=['Jurisdiction_Region', 'counts'], ascending=[True, False])

# Get a list of unique regions
regions = df['Jurisdiction_Region'].unique()

# Create a color map with a distinct color for each region
color_map = plt.get_cmap('tab20')(range(len(regions)))
region_colors = {region: color for region, color in zip(regions, color_map)}

# Map the color to each jurisdiction based on its region
df['color'] = df['Jurisdiction_Region'].map(region_colors)

# Group by region to get total counts per region
region_counts = df.groupby('Jurisdiction_Region')['counts'].sum().reset_index(name='region_counts').sort_values(by=['Jurisdiction_Region', 'region_counts'], ascending=[True, True])


# Group by region and jurisdiction to get counts for each jurisdiction
jurisdiction_counts = df.groupby(['Jurisdiction_Region', 'Jurisdiction_name']).sum().reset_index().sort_values(by=['Jurisdiction_Region', 'counts'], ascending=[True, False])


# Sort the jurisdiction_counts to ensure matching colors in the plot
jurisdiction_counts = jurisdiction_counts.sort_values(by='Jurisdiction_Region')


# Set the threshold for displaying labels on the outer pie chart
threshold = 4  # You can adjust this number as needed for your dataset

# Start plotting
fig, ax = plt.subplots(figsize=(18, 20), subplot_kw=dict(aspect="equal"))

# Inner Pie Chart: Regions
inner_pie_wedges, _ = ax.pie(
    region_counts['region_counts'],
    radius=.8,
    labeldistance=0.5,
    colors=[region_colors[region] for region in region_counts['Jurisdiction_Region']],
    wedgeprops=dict(width=0.7, edgecolor='w')
)

# Apply labels to the inner pie chart for regions
for wedge, label in zip(inner_pie_wedges, region_counts['Jurisdiction_Region']):
    angle = (wedge.theta2 - wedge.theta1) / 2 + wedge.theta1
    x = 0.5 * np.cos(np.deg2rad(angle))
    y = 0.5 * np.sin(np.deg2rad(angle))
    ax.text(x, y, label, horizontalalignment='center', verticalalignment='center', size=12, weight="bold")

# Outer Pie Chart: Jurisdiction names
outer_pie_wedges, _ = ax.pie(
    jurisdiction_counts['counts'],
    radius=1.2,
    colors=[region_colors[region] for region in jurisdiction_counts['Jurisdiction_Region']],
    wedgeprops=dict(width=0.6, edgecolor='w')
)

# Annotate outer pie chart slices
for i, (wedge, label) in enumerate(zip(outer_pie_wedges, jurisdiction_counts['Jurisdiction_name'])):
    ang = (wedge.theta2 - wedge.theta1) / 2. + wedge.theta1
    x = 1. * np.cos(np.deg2rad(ang))
    y = 1. * np.sin(np.deg2rad(ang))

    if jurisdiction_counts['counts'].iloc[i] > threshold:
        horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
        connectionstyle = f"angle,angleA=0,angleB={ang}"
        ax.annotate(label, xy=(x, y), xytext=(1.4 * np.sign(x), 1.21 * y),
                    arrowprops=dict(arrowstyle="->", connectionstyle=connectionstyle),
                    horizontalalignment=horizontalalignment, size=20)

ax.set_title('Nested Pie Chart: Jurisdiction Counts by Region', fontsize=25, pad=20)

plt.show()

## Stacked Trend of Jurisdiction Contributions by Region

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import itertools

# Your DataFrame is assumed to be named 'df'.
# Number of jurisdictions
num_colors = len(data['Jurisdiction_Region'].unique())

# Combine multiple color maps
cmaps = [plt.cm.tab20, plt.cm.Set3, plt.cm.Paired]  # Add more if needed
colors = list(itertools.chain.from_iterable([cmap(np.linspace(0, 1, cmap.N)) for cmap in cmaps]))

# If more colors are needed, generate custom colors
while len(colors) < num_colors:
    # Generate a random color
    random_color = mcolors.to_rgba(np.random.rand(4,))
    # Check for distinctiveness and add to the list
    if all(mcolors.colorConverter.to_rgb(random_color) != mcolors.colorConverter.to_rgb(existing_color) for existing_color in colors):
        colors.append(random_color)

# Ensure all data is numeric, converting 'Application Year' to a numeric type if necessary
data['Application Year'] = pd.to_numeric(data['Application Year'], errors='coerce')

# Drop rows with NaN in 'Application Year' after conversion
data.dropna(subset=['Application Year'], inplace=True)

# Group by 'Application Year' and 'Jurisdiction_Region', count the occurrences, and reset index
trend_data = data.groupby(['Application Year', 'Jurisdiction_Region']).size().reset_index(name='Counts')

# Pivot the data to have 'Application Year' as the index and 'Jurisdiction_Region' as the columns with their respective counts
trend_pivot = trend_data.pivot(index='Application Year', columns='Jurisdiction_Region', values='Counts').fillna(0)

# Convert the 'Application Year' index back to integer if it was changed to float during the process
trend_pivot.index = trend_pivot.index.astype(int)

# Sort the columns by their overall sum so that the largest area is at the bottom
sorted_regions = trend_pivot.sum().sort_values(ascending=False).index
trend_pivot = trend_pivot[sorted_regions]

# Plotting
plt.figure(figsize=(12, 6))

# Stackplot does the cumulative plot in one go, no need to cumsum manually
plt.stackplot(trend_pivot.index, trend_pivot.T, labels=trend_pivot.columns, colors = colors[:num_colors], alpha=0.8)

# Add title and labels
plt.title('Stacked Trend of Jurisdiction Contributions by Region', fontsize=16)
plt.xlabel('Application Year', fontsize=14)
plt.ylabel('Number of Patent Families', fontsize=14)

# Add legend
plt.legend(title='Region', loc='upper left') #, bbox_to_anchor=(1.05, 1))

# Show grid
plt.grid(True)

# Display the plot
plt.tight_layout()  # Adjust the layout to prevent clipping
plt.show()


## Stacked Trend of Jurisdiction Contributions by Region (Percentage)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import itertools

# Your DataFrame is assumed to be named 'data'.
# Calculate total counts for each jurisdiction
total_counts = data.groupby('Jurisdiction_Region').size()

# Sort jurisdictions by total counts
sorted_regions = total_counts.sort_values(ascending=False).index

# Number of jurisdictions
num_colors = len(data['Jurisdiction_Region'].unique())

# Combine multiple color maps
cmaps = [plt.cm.tab20, plt.cm.Set3, plt.cm.Paired]  # Add more if needed
colors = list(itertools.chain.from_iterable([cmap(np.linspace(0, 1, cmap.N)) for cmap in cmaps]))

# If more colors are needed, generate custom colors
while len(colors) < num_colors:
    # Generate a random color
    random_color = mcolors.to_rgba(np.random.rand(4,))
    # Check for distinctiveness and add to the list
    if all(mcolors.colorConverter.to_rgb(random_color) != mcolors.colorConverter.to_rgb(existing_color) for existing_color in colors):
        colors.append(random_color)

# Ensure all data is numeric, converting 'Application Year' to a numeric type if necessary
data['Application Year'] = pd.to_numeric(data['Application Year'], errors='coerce')

# Drop rows with NaN in 'Application Year' after conversion
data.dropna(subset=['Application Year'], inplace=True)

# Group by 'Application Year' and 'Jurisdiction_Region', count the occurrences, and reset index
trend_data = data.groupby(['Application Year', 'Jurisdiction_Region']).size().reset_index(name='Counts')

# Pivot the data to have 'Application Year' as the index and 'Jurisdiction_Region' as the columns with their respective counts
trend_pivot = trend_data.pivot(index='Application Year', columns='Jurisdiction_Region', values='Counts').fillna(0)

# Convert counts to percentages of the row total (i.e., total counts for each year)
trend_pivot = trend_pivot.div(trend_pivot.sum(axis=1), axis=0) * 100

# Convert the 'Application Year' index back to integer if it was changed to float during the process
trend_pivot.index = trend_pivot.index.astype(int)

# Sort the columns by their overall sum so that the largest region is at the bottom
#sorted_regions = trend_pivot.sum().sort_values(ascending=False).index
trend_pivot = trend_pivot[sorted_regions]

# Plotting
plt.figure(figsize=(12, 6))

# Stackplot now plots percentages
plt.stackplot(trend_pivot.index, trend_pivot.T, labels=trend_pivot.columns, colors=colors[:num_colors], alpha=0.8)

# Add title and labels
plt.title('Stacked Trend of Jurisdiction Contributions by Region (Percentage)', fontsize=18, fontweight='bold')
plt.xlabel('Application Year', fontsize=16, fontweight='bold')
plt.ylabel('Percentage of Patent Families', fontsize=16, fontweight='bold')

# Add legend
plt.legend(title='Region', loc='upper left') #, bbox_to_anchor=(1.05, 1))

# Show grid
plt.grid(True)

# Display the plot
plt.tight_layout()  # Adjust the layout to prevent clipping
plt.show()


In [ ]:
trend_pivot.replace(0, '')

## Yearly Trend of Patents Families for Leading Countries ////// 'Yearly Trend of Patents Families for Leading Countries

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Assuming data is already loaded...

# Grouping patents by country
country_counts = data['Jurisdiction_name'].value_counts()

# Plotting the top 10 countries using seaborn
plt.figure(figsize=(14, 7))
sns.barplot(x=country_counts.head(10).index, y=country_counts.head(10).values, palette='viridis')
plt.title('Top 10 Countries Leading in Phage Therapy Innovations', fontsize=18, fontweight='bold')
plt.xlabel('Country', fontsize=16, fontweight='bold')
plt.ylabel('Number of Patents Families', fontsize=16, fontweight='bold')
plt.xticks(rotation=90, fontsize=14, fontweight='bold')
plt.yticks(fontsize=12)
sns.despine(bottom=True, left=True)
plt.tight_layout()
plt.show()

# Assuming 'Application Date' column exists
data['Year'] = pd.to_datetime(data['Application Date']).dt.year
yearly_counts = data.groupby(['Year', 'Jurisdiction_name']).size().unstack().fillna(0)

# Top 10 countries
top_countries = country_counts.head(10)

# Plotting temporal trends for top countries using seaborn
plt.figure(figsize=(16, 9))
for country in top_countries.index:
    sns.lineplot(data=yearly_counts, x=yearly_counts.index, y=country, label=country)


plt.title('Yearly Trend of Patents Families for Leading Jurisdiction', fontsize=16)
plt.xlabel('Application Year', fontsize=16, fontweight='bold')
plt.ylabel('Number of Patents Families', fontsize=16, fontweight='bold')
plt.legend(title='Jurisdiction', fontsize=12, title_fontsize='14')

plt.xticks(fontsize=14, fontweight='bold')
plt.yticks(fontsize=14, fontweight='bold')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.show()


In [ ]:
country_counts

## Yearly Trend of Patent Families for the Top 10 Jurisdictions

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Assuming data is already loaded...

# Grouping patents by country
country_counts = data['Jurisdiction_name'].value_counts()

# Assuming 'Application Date' column exists and is in the correct date format
data['Year'] = pd.to_datetime(data['Application Date']).dt.year
yearly_counts = data.groupby(['Year', 'Jurisdiction_name']).size().unstack().fillna(0)

# Top 10 countries
top_countries = country_counts.head(10).index

# Plotting temporal trends for top countries using seaborn
# Set up the matplotlib figure
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(25, 10))

# Enumerate over the top countries and create a lineplot for each
for i, country in enumerate(top_countries):
    row = i // 5  # Determine the row of the subplot
    col = i % 5   # Determine the column of the subplot
    ax = axes[row, col]  # Get the subplot axis

    # Plot on the appropriate subplot
    sns.lineplot(ax=ax, data=yearly_counts, x=yearly_counts.index, y=country)
    ax.set_title(country, fontsize=14)
    ax.set_xlabel('Year', fontsize=12)
    if col == 0:  # Only set y-label for the first column to reduce clutter
        ax.set_ylabel('Number of Patent Families', fontsize=12)
    ax.tick_params(labelsize=10)

# Adjusting the layout to prevent overlapping
plt.tight_layout()

# Add a general title
plt.suptitle('Yearly Trend of Patent Families for the Top 10 Jurisdictions', fontsize=18, y=1.05)

# Finally, show the plot
plt.show()


In [ ]:
yearly_counts.replace(0, '')

## Stacked Trend of Jurisdiction Contributions

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import itertools

# Your DataFrame is assumed to be named 'df'.
# Number of jurisdictions
num_colors = len(yearly_counts.columns)

# Combine multiple color maps
cmaps = [plt.cm.tab20, plt.cm.Set3, plt.cm.Paired]  # Add more if needed
colors = list(itertools.chain.from_iterable([cmap(np.linspace(0, 1, cmap.N)) for cmap in cmaps]))

# If more colors are needed, generate custom colors
while len(colors) < num_colors:
    # Generate a random color
    random_color = mcolors.to_rgba(np.random.rand(4,))
    # Check for distinctiveness and add to the list
    if all(mcolors.colorConverter.to_rgb(random_color) != mcolors.colorConverter.to_rgb(existing_color) for existing_color in colors):
        colors.append(random_color)

# Ensure all data is numeric, converting 'Application Year' to a numeric type if necessary
data['Application Year'] = pd.to_numeric(data['Application Year'], errors='coerce')

# Drop rows with NaN in 'Application Year' after conversion
data.dropna(subset=['Application Year'], inplace=True)

# Group by 'Application Year' and 'Jurisdiction_name', count the occurrences, and reset index
trend_data = data.groupby(['Application Year', 'Jurisdiction_name']).size().reset_index(name='Counts')

# Pivot the data to have 'Application Year' as the index and 'Jurisdiction_name' as the columns with their respective counts
trend_pivot = trend_data.pivot(index='Application Year', columns='Jurisdiction_name', values='Counts').fillna(0)

# Convert the 'Application Year' index back to integer if it was changed to float during the process
trend_pivot.index = trend_pivot.index.astype(int)

# Sort the columns by their overall sum so that the largest area is at the bottom
sorted_regions = trend_pivot.sum().sort_values(ascending=False).index
trend_pivot = trend_pivot[sorted_regions]

# Plotting
plt.figure(figsize=(20, 8.5))

# Stackplot does the cumulative plot in one go, no need to cumsum manually
plt.stackplot(trend_pivot.index, trend_pivot.T, labels=trend_pivot.columns, colors=colors[:num_colors], alpha=0.8)

# Add title and labels
plt.title('Stacked Trend of Jurisdiction Contributions', fontsize=16)
plt.xlabel('Application Year', fontsize=14)
plt.ylabel('Number of Patent Families', fontsize=14)

# Add legend
plt.legend(title='Jurisdiction', loc='upper left', bbox_to_anchor=(1.0, 1))

# Show grid
plt.grid(True)

# Display the plot
plt.tight_layout()  # Adjust the layout to prevent clipping
plt.show()


In [ ]:
trend_pivot.replace(0, '')

## Stacked Trend of Jurisdiction Contributions (Percentage)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import itertools

# Calculate total counts for each jurisdiction
total_counts = data.groupby('Jurisdiction_name').size()

# Sort jurisdictions by total counts
sorted_jurisdictions = total_counts.sort_values(ascending=False).index

# Rearrange the columns in 'trend_pivot' based on sorted jurisdictions
trend_pivot = trend_pivot[sorted_jurisdictions]

# Number of jurisdictions
num_colors = len(trend_pivot.columns)

# Combine multiple color maps
cmaps = [plt.cm.tab20, plt.cm.Set3, plt.cm.Paired]  # Add more if needed
colors = list(itertools.chain.from_iterable([cmap(np.linspace(0, 1, cmap.N)) for cmap in cmaps]))

# If more colors are needed, generate custom colors
while len(colors) < num_colors:
    # Generate a random color
    random_color = mcolors.to_rgba(np.random.rand(4,))
    # Check for distinctiveness and add to the list
    if all(mcolors.colorConverter.to_rgb(random_color) != mcolors.colorConverter.to_rgb(existing_color) for existing_color in colors):
        colors.append(random_color)


# Ensure all data is numeric, converting 'Application Year' to a numeric type if necessary
data['Application Year'] = pd.to_numeric(data['Application Year'], errors='coerce')

# Drop rows with NaN in 'Application Year' after conversion
data.dropna(subset=['Application Year'], inplace=True)

# Group by 'Application Year' and 'Jurisdiction_name', count the occurrences, and reset index
trend_data = data.groupby(['Application Year', 'Jurisdiction_name']).size().reset_index(name='Counts')

# Pivot the data to have 'Application Year' as the index and 'Jurisdiction_name' as the columns with their respective counts
trend_pivot = trend_data.pivot(index='Application Year', columns='Jurisdiction_name', values='Counts').fillna(0)

# Convert counts to percentages of the row total (i.e., total counts for each year)
trend_pivot = trend_pivot.div(trend_pivot.sum(axis=1), axis=0) * 100

# Convert the 'Application Year' index back to integer if it was changed to float during the process
trend_pivot.index = trend_pivot.index.astype(int)

# Sort the columns by their overall sum so that the largest region is at the bottom
#sorted_regions = trend_pivot.sum().sort_values(ascending=False).index
trend_pivot = trend_pivot[sorted_jurisdictions]

# Plotting
plt.figure(figsize=(20, 8.5))

# Stackplot now plots percentages
plt.stackplot(trend_pivot.index, trend_pivot.T, labels=trend_pivot.columns, colors=colors[:num_colors], alpha=0.8)

# Add title and labels
plt.title('Stacked Trend of Jurisdiction Contributions (Percentage)', fontsize=18, fontweight='bold')
plt.xlabel('Application Year', fontsize=16, fontweight='bold')
plt.ylabel('Percentage of Patent Families', fontsize=16, fontweight='bold')

# Add legend
plt.legend(title='Jurisdiction', loc='upper left', bbox_to_anchor=(1.0, 1))

# Show grid
plt.grid(True)

# Display the plot
plt.tight_layout()  # Adjust the layout to prevent clipping
plt.show()


In [ ]:
sorted_jurisdictions

In [ ]:
trend_pivot.replace(0, '')

In [ ]:
data[['Lens ID', 'Applicants_regions','Applicants_countries','Applicants_categories']]

# Heatmap graph applicants countries

In [ ]:
from operator import index
def process_data(data, index_cols, split_col, top_n_index, top_n_col):
  # Set the index
  applicants_data = data.set_index(['Lens ID', index_cols])

  # Split and explode the 'Applicants' field
  applicants_list = applicants_data[split_col].str.split(';;').explode().str.strip().to_frame()
      # Reset the index to re-organize data
  applicants_list2 = applicants_list.reset_index()
  applicants_list2.dropna(subset=[index_cols], inplace=True)

  # Set the index
  applicants_list3 = applicants_list2.set_index(['Lens ID', split_col])

  # Split and explode the 'Applicants' field
  applicants_list4 = applicants_list3[index_cols].str.split(';;').explode().str.strip().to_frame()
      # Reset the index to re-organize data
  applicants_list4 = applicants_list4.reset_index()
  applicants_list4[index_cols] = applicants_list4[index_cols].str.title()

  grouped = applicants_list4.groupby([index_cols, split_col]).agg({
      'Lens ID': 'count'
  }).reset_index()

  pivot_df_ipcr_appl = grouped.pivot_table(index=split_col, columns=index_cols, values=['Lens ID'], aggfunc='sum')#['Lens ID']

  pivot_df_ipcr_appl = pivot_df_ipcr_appl.head(top_n_index)

  pivot_df_ipcr_applT = pivot_df_ipcr_appl.T
  # Determine the top 50 companies based on the given metric
  top_50_applicants = pivot_df_ipcr_applT.sum(axis=1).nlargest(top_n_col).index

  # Filter pivot_df to only include these top 50 companies
  pivot_df_ipcr_applT50 = pivot_df_ipcr_applT.loc[top_50_applicants].T

  if '' in pivot_df_ipcr_applT50.index :
    pivot_df_ipcr_applT50 = pivot_df_ipcr_applT50.drop('', axis=0)

  if '' in pivot_df_ipcr_applT50.columns :
    pivot_df_ipcr_applT50 = pivot_df_ipcr_applT50.drop(('Lens ID',               ''), axis=1)

  return pivot_df_ipcr_applT50



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def plot_top_50_heatmap(pivot_df_ipcr, split_col, top_n_index, metric, figsize, xlabel, ylabel):
    """
    Plot a heatmap for the given metric for top 50 companies based on Publication Count.

    Args:
    - pivot_df (pd.DataFrame): The pivoted DataFrame with metrics for each applicant over different application years.
    - metric (str): The metric based on which the top 50 companies are determined. Default is 'Publication Count'.

    Returns:
    None
    """
    # Determine the top 50 companies based on the given metric
    top_50_applicants = pivot_df_ipcr[metric].sum(axis=1).nlargest(top_n_index).index

    # Filter pivot_df to only include these top 50 companies
    top_50_data = pivot_df_ipcr.loc[top_50_applicants]

    # Ensure the index is properly formatted
    top_50_data.index = top_50_data.index.to_series().str.title()

    # Round the values and convert to integer
    #top_50_data = np.round(top_50_data).astype(int)

    def convert_to_int(x):
      return int(x) if not pd.isna(x) else np.nan

    for col in top_50_data :
      top_50_data[col] = top_50_data[col].apply(convert_to_int)

    plt.figure(figsize=figsize)

    # Extract data for the given metric
    data_to_plot = top_50_data[metric].reset_index()

    # Wrapping title with year and applicant name
    #data_to_plot['Applicants_categories'] = data_to_plot.apply(lambda row: f"{row['Applicants_categories'].title()}", axis=1)
    data_to_plot[split_col] = data_to_plot.apply(lambda row: f"{row[split_col].title()}", axis=1)

    # Set the Applicants column as the index
    data_to_plot = data_to_plot.set_index(split_col)

    # Create the heatmap
    sns.heatmap(data_to_plot, annot=True, fmt=".0f", cmap='YlGnBu', linewidths=.5, cbar_kws={"orientation": "vertical"})

    plt.xticks(fontsize=14, fontweight='bold')
    plt.yticks(fontsize=14, fontweight='bold')
    #plt.title(f"Heatmap of Top 50 Applicants and the Top 25 IPCR Subclass", fontsize=18, fontweight='bold')
    plt.xlabel(xlabel, fontsize=16, fontweight='bold')
    plt.ylabel(ylabel, fontsize=16, fontweight='bold')
    plt.grid( linestyle='--', alpha=0.7) #axis='x',
    plt.show()

# Example Usage:
# Assuming 'pivot_df' is your DataFrame with the required metrics.
#plot_top_50_heatmap(pivot_df_ipcr_applT50, "Lens ID")
#plot_top_50_heatmap(pivot_df_ipcr_applT50, "Total Citations")
#plot_top_50_heatmap(pivot_df_ipcr_applT50, "Average Citation Count")


##  Applicants categories

In [ ]:
index_cols = 'Applicants_countries'
split_col = 'Applicants_categories'
top_n_col = 50
top_n_index = 4
figsize=(25, 2)
xlabel = "Applicant countries"
ylabel = "Types"

pivot_df = process_data(data, index_cols, split_col, top_n_index, top_n_col)
plot_top_50_heatmap(pivot_df, split_col, top_n_index, "Lens ID", figsize, xlabel, ylabel)

##  Applicants region

In [ ]:
index_cols = 'Applicants_countries'
split_col = 'Applicants_regions'
top_n_col = 50
top_n_index = 10
figsize=(25, 3)
xlabel = "Applicant countries"
ylabel = "Appl. Regions"

df = process_data(data, index_cols, split_col, top_n_index, top_n_col)
plot_top_50_heatmap(df, split_col, top_n_index, "Lens ID", figsize, xlabel, ylabel)

##  Applicants region

In [ ]:
index_cols = 'Applicants_countries'
split_col = 'Applicants'
top_n_col = 50
top_n_index = 50
figsize=(25, 15)
xlabel = "Applicant countries"
ylabel = "Applicants"

df = process_data(data, index_cols, split_col, top_n_index, top_n_col)
plot_top_50_heatmap(df, split_col, top_n_index, "Lens ID", figsize, xlabel, ylabel)

In [ ]:
df.columns

In [ ]:
df.index

In [ ]:
df.drop(('Lens ID',               ''), axis=1)